# 📊 Dashboard Inteligente de Preços - KaBuM!
Este projeto realiza a coleta automatizada de preços de produtos na loja KaBuM utilizando Selenium, armazena os dados em CSV e apresenta visualizações interativas para análise de variação de preços ao longo do tempo.

**Objetivos principais:**
- Automatizar a extração de preços com base em palavras-chave;
- Armazenar os dados em histórico (CSV);
- Analisar e visualizar a evolução dos preços;
- Criar gráficos comparativos para produtos buscados.


In [ ]:
# Instalar dependências e configurar ambiente
!pip install selenium > /dev/null
!apt-get update > /dev/null
!apt install chromium-chromedriver -y > /dev/null
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

# Importações
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# Configuração do navegador invisível
def iniciar_navegador():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(options=chrome_options)

# Função para buscar produtos na Kabum
def buscar_produtos_kabum(palavra_chave):
    nav = iniciar_navegador()
    url = f"https://www.kabum.com.br/busca/{palavra_chave.replace(' ', '-')}"
    nav.get(url)

    produtos = []
    cards = nav.find_elements(By.CLASS_NAME, "productCard")[:5]

    for card in cards:
        try:
            nome = card.find_element(By.CLASS_NAME, "nameCard").text
            preco = card.find_element(By.CLASS_NAME, "priceCard").text
            produtos.append((nome, preco))
        except:
            continue

    nav.quit()
    return produtos

# 🔎 Testar com um exemplo
resultado = buscar_produtos_kabum("notebook")

if resultado:
    print("✅ Produtos encontrados:\n")
    for nome, preco in resultado:
        print(f"📦 {nome}\n💰 {preco}\n")
else:
    print("⚠️ Nenhum produto encontrado.")


In [ ]:
import pandas as pd
from datetime import datetime
import os

def salvar_em_csv(lista_produtos, nome_arquivo="dados_kabum.csv"):
    df = pd.DataFrame(lista_produtos, columns=["nome", "preco"])
    df["data_coleta"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    caminho = f"/content/{nome_arquivo}"

    if os.path.exists(caminho):
        df_existente = pd.read_csv(caminho)
        df = pd.concat([df_existente, df], ignore_index=True)

    df.to_csv(caminho, index=False)
    print(f"📁 Dados salvos com sucesso em: {caminho}")


In [ ]:
salvar_em_csv(resultado)  # "resultado" é a lista que pegamos antes com os produtos


In [ ]:
pd.read_csv("/content/dados_kabum.csv").tail()


## 🧹 Limpeza e Pré-processamento dos Dados

In [ ]:
# Imports necessários
import pandas as pd

# Leitura do CSV
df = pd.read_csv("/content/dados_kabum.csv")

# Converter a coluna de data
df["data_coleta"] = pd.to_datetime(df["data_coleta"], errors="coerce")

# Limpeza da coluna de preços
# Remove 'R$', remove pontos de milhar, troca vírgula por ponto
df["preco"] = df["preco"].replace({'R\$': '', '\.': '', ',': '.'}, regex=True)

# Converte para float
df["preco"] = pd.to_numeric(df["preco"], errors="coerce")

# Mostra os primeiros dados para verificação
df.head()


## 📈 Análise Estatística dos Preços Coletados

In [ ]:
# Estatísticas gerais
df.describe(include='all')


## 📊 Visualização Interativa dos Preços

In [ ]:
import plotly.express as px

# Gráfico de linha por nome de produto ao longo do tempo
fig = px.line(df.sort_values('data_coleta'), x='data_coleta', y='preco', color='nome', title='📈 Evolução de Preços por Produto', markers=True)
fig.update_layout(xaxis_title='Data da Coleta', yaxis_title='Preço (R$)')
fig.show()

In [ ]:
# Últimos preços coletados por produto
ultimos = df.sort_values('data_coleta').groupby('nome').tail(1)
fig_bar = px.bar(ultimos, x='nome', y='preco', title='💰 Últimos Preços Coletados')
fig_bar.update_layout(xaxis_title='Produto', yaxis_title='Preço Atual (R$)', xaxis_tickangle=-30)
fig_bar.show()

## ✅ Conclusão e Próximos Passos
- O dashboard permite monitorar a variação de preços de produtos da KaBuM em tempo quase real.
- Pode ser integrado com notificações de alerta de queda de preço.
- Um próximo passo seria transformar este notebook em uma aplicação web com [Streamlit](https://streamlit.io/) ou [Gradio](https://www.gradio.app/) para uso mais acessível.

**Compartilhamento:** Exporte os gráficos ou salve capturas para uso pessoal ou apresentação.